In [9]:
import yaml

import torch
from neurowhore.nets.sd_net import UWUNet_v1

from IPython.display import Audio

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
 
run_name = 'UWUNet_experiment_1'
cfg_path = f'./cfg/{run_name}/'

checkpoint_dir = './checkpoints/'
model_checkpoint_path = checkpoint_dir + run_name + '_model.pth'

with open(cfg_path + 'model.yaml') as f:
    model_params = yaml.safe_load(f)

In [11]:
#denoiser_model = UWUNet_v1(wav_channels=model_params['wav_channels'],
#                           down_channels=model_params['down_channels'],
#                           up_channels=model_params['up_channels'],
#                           time_emb_dim=model_params['time_emb_dim'],
#                           wn_dilation_depth=model_params['wn_dilation_depth'],
#                           wn_repeats=model_params['wn_repeats']).to(device)

denoiser_model = torch.load(model_checkpoint_path).to(device).eval()

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 7.29 GiB already allocated; 0 bytes free; 7.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [22]:
sr = 16000
batch_size = 1
sample_len = 10000
sample = 0.5 * (torch.rand((batch_size, 1, sample_len), device=device) * 2 - 1)

t_max = model_params['t_max']
t_arange = torch.flip(torch.arange(0, t_max, device=device), (0,))
print(t_arange[0:1])

tensor([199], device='cuda:0')


In [23]:
out = torch.tensor(sample)
with torch.no_grad():
    for tdx in range(t_max):
        tt = t_arange[tdx:tdx+1]
        if tdx % 10 == 0:
            print(tt)
        out = denoiser_model(out, tt)

C:\Users\epirt\AppData\Local\Temp\ipykernel_4136\371609684.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(sample)


tensor([199], device='cuda:0')
tensor([189], device='cuda:0')
tensor([179], device='cuda:0')
tensor([169], device='cuda:0')
tensor([159], device='cuda:0')
tensor([149], device='cuda:0')
tensor([139], device='cuda:0')
tensor([129], device='cuda:0')
tensor([119], device='cuda:0')
tensor([109], device='cuda:0')
tensor([99], device='cuda:0')
tensor([89], device='cuda:0')
tensor([79], device='cuda:0')
tensor([69], device='cuda:0')
tensor([59], device='cuda:0')
tensor([49], device='cuda:0')
tensor([39], device='cuda:0')
tensor([29], device='cuda:0')
tensor([19], device='cuda:0')
tensor([9], device='cuda:0')


In [24]:
Audio(data=out.cpu().numpy()[0, 0, :], rate=sr)